In [1]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
from datetime import datetime
from models.handler import train, test, validate
import pandas as pd
from models.base_model import Model
import numpy as np
import json
from data_loader.forecast_dataloader import ForecastDataset
import torch.utils.data as torch_data
import torch.nn as nn
import time

class Args:
    def __init__(self):
        self.train = True
        self.evaluate = True
        self.dataset = 'ECG_data'
        self.window_size = 12
        self.horizon = 3
        self.train_length = 7
        self.valid_length = 2
        self.test_length = 1
        self.epoch = 50
        self.lr = 1e-4
        self.multi_layer = 5
        self.device = 'cpu'
        self.validate_freq = 1
        self.batch_size = 32
        self.norm_method = 'z_score'
        self.optimizer = 'RMSProp'
        self.early_stop = False
        self.exponential_decay_step = 5
        self.decay_rate = 0.5
        self.dropout_rate = 0.5
        self.leakyrelu_rate = 0.2

args = Args()
print(f'Training configs: {args}')

Training configs: <__main__.Args object at 0x000002943B1FDD30>


In [2]:
data_file = os.path.join('dataset', args.dataset + '.csv')
result_train_file = os.path.join('output', args.dataset, 'train')
result_test_file = os.path.join('output', args.dataset, 'test')
if not os.path.exists(result_train_file):
    os.makedirs(result_train_file)
if not os.path.exists(result_test_file):
    os.makedirs(result_test_file)
data = pd.read_csv(data_file).values

In [3]:
# split data
train_ratio = args.train_length / (args.train_length + args.valid_length + args.test_length)
valid_ratio = args.valid_length / (args.train_length + args.valid_length + args.test_length)
test_ratio = 1 - train_ratio - valid_ratio
train_data = data[:int(train_ratio * len(data))]
valid_data = data[int(train_ratio * len(data)):int((train_ratio + valid_ratio) * len(data))]
test_data = data[int((train_ratio + valid_ratio) * len(data)):]

torch.manual_seed(0)

In [4]:
result_file = result_train_file

node_cnt = train_data.shape[1]
model = Model(node_cnt, 2, args.window_size, args.multi_layer, horizon=args.horizon)
model.to(args.device)
if len(train_data) == 0:
    raise Exception('Cannot organize enough training data')
if len(valid_data) == 0:
    raise Exception('Cannot organize enough validation data')

if args.norm_method == 'z_score':
    train_mean = np.mean(train_data, axis=0)
    train_std = np.std(train_data, axis=0)
    normalize_statistic = {"mean": train_mean.tolist(), "std": train_std.tolist()}
elif args.norm_method == 'min_max':
    train_min = np.min(train_data, axis=0)
    train_max = np.max(train_data, axis=0)
    normalize_statistic = {"min": train_min.tolist(), "max": train_max.tolist()}
else:
    normalize_statistic = None
if normalize_statistic is not None:
    with open(os.path.join(result_file, 'norm_stat.json'), 'w') as f:
        json.dump(normalize_statistic, f)

if args.optimizer == 'RMSProp':
    my_optim = torch.optim.RMSprop(params=model.parameters(), lr=args.lr, eps=1e-08)
else:
    my_optim = torch.optim.Adam(params=model.parameters(), lr=args.lr, betas=(0.9, 0.999))
my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=my_optim, gamma=args.decay_rate)

train_set = ForecastDataset(train_data, window_size=args.window_size, horizon=args.horizon,
                            normalize_method=args.norm_method, norm_statistic=normalize_statistic)
valid_set = ForecastDataset(valid_data, window_size=args.window_size, horizon=args.horizon,
                            normalize_method=args.norm_method, norm_statistic=normalize_statistic)
train_loader = torch_data.DataLoader(train_set, batch_size=args.batch_size, drop_last=False, shuffle=True,
                                        num_workers=0)
valid_loader = torch_data.DataLoader(valid_set, batch_size=args.batch_size, shuffle=False, num_workers=0)

forecast_loss = nn.MSELoss(reduction='mean').to(args.device)

total_params = 0
for name, parameter in model.named_parameters():
    if not parameter.requires_grad: continue
    param = parameter.numel()
    total_params += param
print(f"Total Trainable Params: {total_params}")


Total Trainable Params: 1123303


In [5]:
best_validate_mae = np.inf
validate_score_non_decrease_count = 0
performance_metrics = {}
for epoch in range(args.epoch):
    epoch_start_time = time.time()
    model.train()
    loss_total = 0
    cnt = 0
    for i, (inputs, target) in enumerate(train_loader):
        inputs = inputs.to(args.device)
        target = target.to(args.device)
        model.zero_grad()
        forecast, _ = model(inputs)
        loss = forecast_loss(forecast, target)
        cnt += 1
        loss.backward()
        my_optim.step()
        loss_total += float(loss)

        
        break

    print('| end of epoch {:3d} | time: {:5.2f}s | train_total_loss {:5.4f}'.format(epoch, (
            time.time() - epoch_start_time), loss_total / cnt))
    
    break
        

    # save_model(model, result_file, epoch)
    
    if (epoch+1) % args.exponential_decay_step == 0:
        my_lr_scheduler.step()
    if (epoch + 1) % args.validate_freq == 0:
        is_best_for_now = False
        print('------ validate on data: VALIDATE ------')
        performance_metrics = \
            validate(model, valid_loader, args.device, args.norm_method, normalize_statistic,
                        node_cnt, args.window_size, args.horizon,
                        result_file=result_file)
        if best_validate_mae > performance_metrics['mae']:
            best_validate_mae = performance_metrics['mae']
            is_best_for_now = True
            validate_score_non_decrease_count = 0
        else:
            validate_score_non_decrease_count += 1
        # save model
        # if is_best_for_now:
        #     save_model(model, result_file)
    # early stop
    if args.early_stop and validate_score_non_decrease_count >= args.early_stop_step:
        break

Input size: torch.Size([32, 12, 140])
reformat: torch.Size([140, 32, 12])
After GRU torch.Size([140, 32, 140])
After reformat torch.Size([32, 140, 140])
into graph attention:  torch.Size([32, 140, 140])
reformat torch.Size([32, 140, 140])
key=Input*WeightKey size torch.Size([32, 140, 1]) = <built-in method size of Tensor object at 0x0000029421045168> torch.Size([140, 1])
query=input*weightQuery size torch.Size([32, 140, 1]) = torch.Size([32, 140, 140]) torch.Size([140, 1])
define data = key.repeat(1, 1, N).view(bat, N * N, 1) + query.repeat(1, N, 1) torch.Size([32, 19600, 1]) torch.Size([32, 19600, 1]) torch.Size([32, 19600, 1])
squeeze data: torch.Size([32, 19600])
view data torch.Size([32, 140, 140])
pass leakyrelu torch.Size([32, 140, 140])
compute attention torch.Size([32, 140, 140])
dropout on attention:  torch.Size([32, 140, 140])
Attention mean'd on dim 0 torch.Size([140, 140])
Degree = attention sum on dim 1 torch.Size([140])
attention refactor torch.Size([140, 140])
degree_L t

AttributeError: 'NoneType' object has no attribute 'size'

In [8]:
rnn = nn.GRU(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)


input.size(), output.size(), hn.size()

(torch.Size([5, 3, 10]), torch.Size([5, 3, 20]), torch.Size([2, 3, 20]))